# **k-NN**

Dejando a un lado por el momento los algoritmos de aprendizaje supervisado, veamos el método k-NN, k Vecinos Más Cercanos o en inglés ***k-Nearest Neighbors***. Se trata de una herramienta de clasificación realmente simple.

# **Distancias entre vecinos**

Pongamos una situación en la que quisieramos saber a qué clase pertenece una instancia de nuestros datos. Cada instancia es un vector con tantas dimensiones como columnas. En caso de que contemos con datos que han sido correctamente clasificados, simplemente podríamos medir la distancia entre este hipotético vector y el resto de instancias clasificadas. Si un porcentaje alto de los datos cercanos pertenecen a una determinada clase, no es difícil inferir que nuestro vector hipotético pertenece a la misma clase. Esto es lo que hace k-NN: asigna una clase o categoría en función de la clase a la que pertenecen los vectores vecinos.

En la gran mayoría de los casos utilizaremos la distancia euclídea para calcular la distancia entre vectores. La fórmula general para la distancia entre dos vectores es:

$$d(v_{1},v_{2}) =  \sqrt{\sum_{i=1}^{n}(v_{1i}-v_{2i})^{2}}$$



<div style="display: flex; justify-content: center;">
<img src="https://drive.google.com/uc?export=view&id=1UFVDVmPkb0hrlk8J-DcY3viNxl4aCKfy" width="800">


La asignación a una clase u otra depende de la proporción de vecinos pertenecientes a cada clase. Y esto a su vez es resultado de cómo definamos "vecino". El algoritmo k-NN espera que le indiquemos el número $k$ de los vecinos más cercanos a chequear. Evidentemente, menos vecinos implicará un algoritmo más restrictivo, lo cual supone más precisión, pero también mayor sensibilidad al ruido inherente de los datos. Ampliando $k$ podremos reducir la precisión a costa de estabilidad.

Una vez calculado la distancia entre el vector y el resto de datos etiquetados y seleccionado aquellos más cercanos, la asignación a una clase puede seguir simplemente el criterio de la clase más frecuente entre los vecinos. También es posible realizar una media si son valores numéricos.

La sencillez de k-NN supone que, en vez de ser utilizado en tareas de clasificación, suele verse como herramienta de imputación de datos faltantes. Así es como lo veremos en el ejemplo de abajo.

# **Imputación de datos con k-NN**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Cargamos el dataset
data = pd.read_csv("/content/sample_data/california_housing_train.csv")
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [ ]:
columns_name = data.columns.tolist()

In [ ]:
# CREAMOS UN DATASET CON VALORES NULOS
np.random.seed(42)
missing_indexes = np.random.randint(data.shape[0], size = 20)
original_values = data.loc[missing_indexes, "median_house_value"]
original_values

15795    236100.0
860      142600.0
5390     147400.0
11964    115300.0
11284    134600.0
6265     189400.0
16850     87500.0
4426     171400.0
14423    500001.0
11363     77100.0
16023    391900.0
8322     158500.0
1685     112500.0
769      125700.0
2433     361400.0
5311     500000.0
5051     282600.0
6420     207000.0
6396      97500.0
8666     223800.0
Name: median_house_value, dtype: float64

In [ ]:
data.loc[missing_indexes,"median_house_value"] = np.nan

In [ ]:
data.isnull().sum()

longitude              0
latitude               0
housing_median_age     0
total_rooms            0
total_bedrooms         0
population             0
households             0
median_income          0
median_house_value    20
dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler

# RESCALADO DE DATOS
scaler = StandardScaler()

# Como algoritmo basado en distancias que es, es conveniente realizar k-NN con datos escalados.
predict_data = data.drop("median_house_value", axis=1)
predict_data = scaler.fit_transform(predict_data)
predict_data = pd.DataFrame(predict_data, columns=data.columns[:-1])
predict_data["median_house_value"] = data["median_house_value"]
predict_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,2.619365,-0.671520,-1.079671,1.361695,1.764204,-0.361184,-0.075998,-1.252543,66900.0
1,2.539569,-0.573264,-0.761872,2.296608,3.230441,-0.261865,-0.099404,-1.081483,80100.0
2,2.494683,-0.905463,-0.920772,-0.882462,-0.866956,-0.955354,-0.999252,-1.170105,85700.0
3,2.489696,-0.928857,-1.159121,-0.524186,-0.480230,-0.796793,-0.715774,-0.362600,73400.0
4,2.489696,-0.961609,-0.682422,-0.545747,-0.506328,-0.701830,-0.622148,-1.026454,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-2.342963,2.318265,1.859971,-0.195728,-0.344995,-0.455275,-0.343872,-0.799999,111400.0
16996,-2.347950,2.369733,0.588774,-0.135174,-0.027073,-0.205236,-0.094203,-0.715727,79000.0
16997,-2.362912,2.907801,-0.920772,0.015292,-0.019955,-0.161675,-0.117609,-0.446663,103600.0
16998,-2.362912,2.889085,-0.761872,0.012999,0.029868,-0.114630,-0.060394,-0.997787,85800.0


In [ ]:
from sklearn.impute import KNNImputer

# IMPUTACIÓN CON K-NN
imputer = KNNImputer(n_neighbors=1, weights="distance", metric="nan_euclidean") # Con weights="distance" haremos que los vecinos más cercanos cuenten más en la predicción.
predict_data = imputer.fit_transform(predict_data)
predict_data = pd.DataFrame(predict_data, columns=columns_name)


In [ ]:
predict_data.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [ ]:
print(sum(predict_data.loc[missing_indexes, "median_house_value"] - original_values))

32199.00000000006
